In [ ]:
import os
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import cv2

In [ ]:
import tensorflow as tf

In [ ]:
base_path = "../input/cassava-leaf-disease-classification/"

with open(os.path.join(base_path, 'label_num_to_disease_map.json'), 'r') as f:
    class_map = json.load(f)
    class_map = {int(k):v for k,v in class_map.items()}
print(class_map)

In [ ]:
print("Number of Images in Train Set: {}".format(len(os.listdir(os.path.join(base_path, 'train_images')))))

In [ ]:
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
train_df

In [ ]:
global_seed = 23
import random
random.seed(global_seed)

In [ ]:
disease_names = open(base_path+'label_num_to_disease_map.json')
disease_names = json.load(disease_names)
train_df['disease_name'] = train_df['label'].apply(lambda x: disease_names[str(x)])
# credit: https://www.kaggle.com/ramjib/cassava-leaf-disease-eda-and-outliers
train_df

In [ ]:
my_colors = 'kckckc'
plt.bar(x=train_df['disease_name'].value_counts().index, height=train_df['disease_name'].value_counts().values, color =my_colors)
plt.xticks(rotation=90)
plt.show()

In [ ]:
def plot_batch(data=df):
    plt.figure(figsize=(16,12))
    for i in range(9):
        k = np.random.randint(0, len(data)) #for plotting random images from dataset
        image = cv2.imread(os.path.join(base_path, 'train_images/', data.image_id[k]))
        
        plt.subplot(3,3,i+1)
        plt.imshow(image)
        plt.axis("off")
        plt.title("Class Label:{}\nClass Name:{}".format(data.label[k], data.disease_name[k]))
    
    plt.tight_layout()
    plt.show()
# Credit - https://www.kaggle.com/anantgupt/cassava-leaf-doctor-eda-keras/notebook#5:-MODEL-TRAINING-AND-DEFINING-CALLBACKS

In [ ]:
plot_batch(train_df)

In [ ]:
for i in range(5):
    temp_df = train_df.loc[df['label']==i]
    temp_df.reset_index(inplace=True)
    print("Class label:", i)
    plot_batch(temp_df)

# Baseline model with FastAI
Most of the following is adapted from kaggle.com/muellerzr/cassava-fastai-starter

In [ ]:
from fastai.vision.all import *
set_seed(23)

In [ ]:
path = Path("../input")
data_path = path/'cassava-leaf-disease-classification'
data_path.ls()

In [ ]:
df = train_df
df['image_id'] = df['image_id'].apply(lambda x: f'train_images/{x}')

In [ ]:
df.head()

## Building the `DataBlock`


In [ ]:
blocks = (ImageBlock, CategoryBlock)
splitter = RandomSplitter(valid_pct=0.2)

In [ ]:
def get_x(row): return data_path/row['image_id']

def get_y(row): return row['label']

In [ ]:
item_tfms = [Resize(448)]
batch_tfms = [RandomResizedCropGPU(224), *aug_transforms(), Normalize.from_stats(*imagenet_stats)]

In [ ]:
block = DataBlock(blocks = blocks,
                 get_x = get_x,
                 get_y = get_y,
                 splitter = splitter,
                 item_tfms = item_tfms,
                 batch_tfms = batch_tfms)

In [ ]:
dls = block.dataloaders(df, bs=64)

We're checking a batch of data to make sure everything looks alright:

In [ ]:
dls.show_batch(figsize=(12,12))

Looks shipshape!

## Training this FastAI model

The code below is from tanlikesmath's notebook (via kaggle.com/muellerzr/cassava-fastai-starter). We have to add a resnet model to our directory (see https://forums.fast.ai/t/how-can-i-load-a-pretrained-model-on-kaggle-using-fastai/13941/24 for instructions) before running the cell below. The code below moves our pretrained weights to where fastai will expect it:

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'

In [ ]:
learn = cnn_learner(dls, resnet50, opt_func=ranger, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)

In [ ]:
def fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tune with `freeze` for `freeze_epochs` then with `unfreeze` from `epochs` using discriminative LR"
    self.freeze()
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()
    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)

In [ ]:
@patch
def fine_tune_flat(self:Learner, epochs, base_lr=4e-3, freeze_epochs=1, lr_mult=100, pct_start=0.75, 
                   first_callbacks = [], second_callbacks = [], **kwargs):
    "Fine-tune applied to `fit_flat_cos`"
    self.freeze()
    self.fit_flat_cos(freeze_epochs, slice(base_lr), pct_start=0.99, cbs=first_callbacks, **kwargs)
    base_lr /= 2
    self.unfreeze()
    self.fit_flat_cos(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, cbs=second_callbacks)

In [ ]:
learn.lr_find()
# Interrupted b/c taking too long; see Mueller's notebook for output
# His SuggestedLRs were: (lr_min=0.017378008365631102, lr_steep=0.14454397559165955)

We'll choose a learning rate of roughly 4e-3 to start.

In [ ]:
cbs1 = [MixUp(alpha = 0.7)]
cbs2 = [MixUp(alpha = 0.3)]

We're training for 1 epoch frozen and 2 unfrozen, with a `pct_start` of 0.72:

In [ ]:
# Note that the original had `start_pct`, but it should have been `pct_start`
learn.fine_tune_flat(2, base_lr=1e-3, pct_start=0.72, first_callbacks=cbs1, second_callbacks=cbs2)

## Submitting some results

In [ ]:
sample_df = pd.read_csv(data_path/'sample_submission.csv')
sample_df.head()

In [ ]:
sample_copy = sample_df.copy()
sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: f'test_images/{x}')

In [ ]:
test_dl = learn.dls.test_dl(sample_copy)

We'll look at a batch of data to make sure it all looks okay:

In [ ]:
test_dl.show_batch()

Next we'll get some predictions. We will use the `.tta` method to run test-time-augmentation which can help boost our accuracy some:

In [ ]:
preds, _ = learn.tta(dl=test_dl)

Then we submit them.

In [ ]:
sample_df['label'] = preds.argmax(dim=-1).numpy()
sample_df.to_csv('submission.csv',index=False)